In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import scipy.sparse as sp
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
import pickle
import math
import pandas as pd
import pickle
import math
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler



/Users/szg/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.linear_model.logistic module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
obj_data_dir = "data/"
def save_obj(obj, name ):
    with open(obj_data_dir + name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open(obj_data_dir + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [3]:
import pandas as pd
BasicFeatures = load_obj('pre_features_v2')
# in-degree statistics
df = pd.DataFrame(BasicFeatures)
df =  df.T
df.rename(columns={0: 'num_of_neighbours',
                   1: 'log_num_of_neighbours',
                  2: 'list_of_neighbours',
                  3: 'list_of_in_neighbours',
                  4: 'num_of_in_neighbours',
                  5: 'list_of_out_neighbours',
                  6: 'num_of_out_neighbours'}, inplace=True)

In [4]:
df.loc[[2]]

,num_of_neighbours,log_num_of_neighbours,list_of_neighbours,list_of_in_neighbours,num_of_in_neighbours,list_of_out_neighbours,num_of_out_neighbours
2,1,1.4427,[3469653],[3469653],1,[],0


In [5]:
'''
接下来生成edges_of_all_test_data_related

首先需要提取test.public 中所有的点，存储为 nodes_in_test_data

然后根据nodes_in_test_data 生成 edges_of_all_test_nodes_related
'''

with open("data/test-public.txt", "r") as f:
     test_data = f.readlines()

# 提取test_data 后删除第一行的ID
test_data = test_data[1:]
# 测试
Edges = load_obj('Edges')
print(Edges[:10])
print(test_data[1].split())

[(540762, 1912140), (540762, 1537559), (540762, 3091331), (540762, 2757277), (540762, 3237295), (540762, 1070876), (540762, 4008078), (540762, 1824878), (540762, 1005927), (540762, 2703564)]
['2', '2052043', '1401960']


In [6]:
# 提取test_data 中所有点
nodes_in_test_data = []
for line in test_data:
    line = line.split()
    source = line[1]
    sink = line[2]
    nodes_in_test_data.append(int(source))
    nodes_in_test_data.append(int(sink))


temp = nodes_in_test_data
temp_set = set(temp)
temp = list(temp_set)
nodes_in_test_data = temp
# 生成出nodes_in_test_data 后用hash set去重
# 可以发现大概有50个点是重复的
print(len(nodes_in_test_data), len(temp))
print(nodes_in_test_data[:10])

3948 3948
[2629632, 2113536, 1900547, 24581, 2146310, 1187846, 3227660, 892941, 1884174, 3350546]


In [7]:
hash_test_nodes = set(nodes_in_test_data)


In [20]:
from tqdm import tqdm 
edges_of_all_test_nodes_related = []
nodes_related = []
for line in tqdm(Edges):
    (node1, node2) = line
    if node1 in hash_test_nodes:
        nodes_related.append(node1)
    if node2 in hash_test_nodes:
        nodes_related.append(node2)
    if (node1 in hash_test_nodes) or (node2 in hash_test_nodes):
        edges_of_all_test_nodes_related.append((node1, node2))

100%|██████████| 24004361/24004361 [00:09<00:00, 2421723.49it/s]


In [21]:
print('edges_related: ',len(edges_of_all_test_nodes_related))
print(edges_of_all_test_nodes_related[:10])
nodes_related = set(nodes_related)
print('\nnodes related: ', len(nodes_related))


edges_related:  5068485
[(540762, 1912140), (540762, 1537559), (540762, 3091331), (540762, 2757277), (540762, 3237295), (540762, 1070876), (540762, 4008078), (540762, 1824878), (540762, 1005927), (540762, 2703564)]

nodes related:  3948


In [25]:
save_obj(edges_of_all_test_nodes_related, 'edges_of_all_test_nodes_related')